In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import numpy as np
import os.path
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from sklearn.utils import shuffle

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)
global_step = tf.Variable(0, name='global_step', trainable=False)

In [4]:
# Import data
COLUMNS = ["media_type", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size","print_time_sec"]
FEATURES = ["media_type", "media_waste", "mode_name", "print_height",
           "print_width","print_id", "R", "G", "B","width","height","w/h","size"]
LABEL = "print_time_sec"

data_set = pd.read_csv("data_sheet.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS).astype(np.float32)

In [5]:
dummy_fields = ['media_type', 'mode_name','print_id']
for each in dummy_fields:
    dummies = pd.get_dummies(data_set[each], prefix=each, drop_first=False)
    data_set = pd.concat([data_set, dummies], axis=1)
    
fields_to_drop = ['media_type', 'mode_name','print_id']
data = data_set.drop(fields_to_drop, axis=1)
data.to_csv('out.csv')

In [6]:
time = data_set['print_time_sec']
data = data.drop('print_time_sec', axis=1)
data = pd.concat([data, time], axis=1)
data.to_csv('out.csv')
data.shape

(21672, 24)

In [7]:
FEATURES =['print_height', 'print_width', 'R', 'G', 'B', 'width', 'height',
       'w/h', 'size', 'media_type_1.0', 'media_type_2.0',
       'media_type_3.0', 'media_type_4.0', 'media_type_5.0',
       'mode_name_1.0', 'mode_name_2.0', 'mode_name_3.0', 'mode_name_4.0',
       'print_id_1.0', 'print_id_2.0', 'print_id_3.0', 'print_id_4.0',
       'print_id_5.0']

In [8]:
#cut data to three sets
def train_split(data):
    data = shuffle(data)
    total=data.shape[0]
    # split data  to test data 5%
    test = data[-int(0.05*total):]
    # Now remove the test data from the data set 
    data = data[:-int(0.05*total)]
    # split data  to valid data 2%
    valid = data[-int(0.01*total):]
    # Now remove the test data from the data set 
    data = data[:-int(0.01*total)]
    
    return data,test,valid

train,test,valid=train_split(data)

In [9]:
print (valid.shape,test.shape,train.shape)
train.to_csv('train.csv')
test.to_csv('test.csv')
valid.to_csv('valid.csv')

(216, 24) (1083, 24) (20373, 24)


In [10]:
def train_fn(data_set):
    #data_set = (data_set - data_set.mean()) / (data_set.max() - data_set.min())
    data_set = (data_set - data_set.mean()) / data_set.std()
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels
def test_fn(data_set):
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels
def valid_fn(data_set):
    feature_cols = data_set[FEATURES]
    labels = data_set[LABEL]
    return feature_cols, labels

train_x, train_y = train_fn(train)
test_x, test_y = test_fn(test)
valid_x, valid_y = valid_fn(valid)

In [11]:
# Parameters
learning_rate = 0.1
training_iters = 50
batch_size = 150

# Network Parameters
n_input = 23 # MNIST data input (img shape: 28*28)
n_classes = 1 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units
model_path = "network/my-model"

x = tf.placeholder(tf.float32, [None, n_input]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32) # 0-9 digits recognition => 10 classes

In [12]:
train_x=train_x.values
train_y=train_y.values
test_x=test_x.values
test_y=test_y.values
valid_x=valid_x.values
valid_y=valid_y.values
train_x = tf.constant(train_x)
train_y = tf.constant(train_y)
test_x = tf.constant(test_x)
test_y = tf.constant(test_y)
valid_x = tf.constant(valid_x)
valid_y = tf.constant(valid_y)

In [13]:
# Create model
def network(x, weights, biases, dropout):
    
    # Fully connected layer
    fc1 = tf.add(tf.matmul(x, weights['wc1']), biases['bc1'])
    #fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    #fc1 = tf.nn.dropout(fc1, dropout)
    
    fc1 = tf.add(tf.matmul(fc1, weights['wc2']), biases['bc2'])
    #fc1 = tf.nn.relu(fc1)
    
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([n_input, 10])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([10, 10])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([10, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([10])),
    'bc2': tf.Variable(tf.random_normal([10])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [14]:
# Construct model
pred = network(x, weights, biases, dropout)

# Define loss and optimizer
cost = tf.reduce_mean(tf.losses.mean_squared_error(predictions=pred, labels=y))

# Evaluate model
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 

# Predictions for the validation, and test data.

test_loss = tf.reduce_mean(tf.losses.mean_squared_error(predictions=pred, labels=y))

valid_pred = network(x, weights, biases, dropout)

In [15]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [ ]:
# Launch the graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print("Initialized")
  saver = tf.train.import_meta_graph('network/my-model.meta')
  saver.restore(sess, tf.train.latest_checkpoint('network/'))   
  #print("Model restored from file: %s" % save_path)
  # get current time
  t1 = datetime.datetime.now()
  for step in range(training_iters):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) %(int(train_x.shape[0]) - batch_size)
    # Generate a minibatch. 
    batch_data = train_x[offset:(offset + batch_size)].eval()
    batch_labels = train_y[offset:(offset + batch_size)].eval()
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {x : batch_data, y : batch_labels}
    _, l = sess.run([optimizer, cost], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Loss at step %d: %f" % (step, l))
  global_step+=training_iters 
  print('global_step: %s' % tf.train.global_step(sess, global_step))
  t2 = datetime.datetime.now()
  # print runtime
  print ('runtime:',t2-t1)
  # Save model weights to disk
  save_path = saver.save(sess, model_path,global_step=global_step)
  saver.export_meta_graph(model_path + '.meta') 
  print("Model saved in file: %s" % save_path)

Initialized
INFO:tensorflow:Restoring parameters from network/my-model-50
Loss at step 0: 1364609.750000
Loss at step 10: 39224.867188
Loss at step 20: 15619.988281
Loss at step 30: 12846.159180
Loss at step 40: 1057.232422
global_step: 50
runtime: 0:07:44.229830


In [ ]:
# Launch the graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print("Initialized")
  saver = tf.train.import_meta_graph('network/my-model.meta')
  saver.restore(sess, tf.train.latest_checkpoint('network/'))   
  print("Test loss %f",sess.run([test_loss], feed_dict={x:test_x[:].eval(), y:test_y[:].eval()}))
  #for step in range(int(valid_x.shape[0])):
  pred = sess.run([valid_pred], feed_dict={x:valid_x[:].eval()})
  print("Prediction: %d",(pred))

Initialized


In [ ]:
pred=np.asarray(pred)
pred = pred.reshape(216,1)
pred.shape